In [1]:
#!pip install recordlinkage==0.14

import recordlinkage
import pandas as pd
import recordlinkage as rl 
from recordlinkage.index import Block
from recordlinkage.algorithms.distance import _haversine_distance
from recordlinkage.algorithms.numeric import _linear_sim
import random
import pickle 


In [2]:
#Reading Data
df_clean = pd.read_csv(r'C:\Users\Invitado\Documents\Python\DS_MASTER\City_Adam_Intern\df_red.csv')

#There was a duplicated entry in ListingId so delete
df_clean[df_clean.duplicated(subset=['ListingId']) == True]
df_clean.drop(132,axis=0,inplace=True)
df_clean[df_clean['ListingId']==10217568]


#To have tuples with LitingsId as index ,useful for later processing
df_tuple = df_clean.set_index('ListingId')

In [3]:
# To compare Harversine we need lat and ln in a tuple
df_clean['Lat_Lng'] =list(zip(df_clean.Lat, df_clean.Lng))

In [4]:
#Blocking of data , feature Neighborhood
indexer = rl.BlockIndex(on='Neighborhood')
candidate_links = indexer.index(df_tuple)

print ('Candidate links:',len(candidate_links))

Candidate links: 3470454


In [5]:
#Further blocking include longitude and latitude
compare_cl = rl.Compare()

compare_cl.exact('Neighborhood','Neighborhood', label='Neighborhood')
compare_cl.geo('Lat','Lng','Lat','Lng',method='linear',label='Distance')

features = compare_cl.compute(candidate_links, df_tuple)
features

Neighborhood  Distance
ListingId_1 ListingId_2                        
10025641    10008030                1  0.000000
10027284    10008030                1  0.331520
            10025641                1  0.000000
10043980    10008030                1  0.000000
            10025641                1  0.972364
...                               ...       ...
42112141    39267402                1  0.000000
49161472    30077792                1  0.000000
            39267402                1  0.000000
            42112141                1  0.000000
40629039    3292084                 1  0.000000

[3470454 rows x 2 columns]

In [6]:
#Select possible matches
matches_blocking = features[features['Distance'] > 0.925] # Threshold used accordig to bnb aprox distance
len(matches_blocking)

44337

In [7]:
#Stratified sampling according to literaure, top 100 most similar , bottom 100 
matches_blocking = matches_blocking.sort_values(by=['Distance'])
top100_matches_blocking = matches_blocking.iloc[-100:]
bottom100_matches_blocking = matches_blocking.iloc[:100]

#list with all the possible matching pairs 150k
top_pairs =list(top100_matches_blocking.index) 
bottom_pairs =list(bottom100_matches_blocking.index) 

In [8]:
#Sampling
sample_top = random.sample(top_pairs,100)
sample_bottom = random.sample(bottom_pairs,100)

In [9]:
df_clean_1 = df_clean.drop(['Host_picture_url','LaBSE_emb', 'Lat_Lng','Host_thumbnail_url'],axis=1)
labels = []

#Labeling loop
for id_1,id_2 in sample_top:
    url_1 =[x for x in df_clean_1['listing_url'][df_clean_1['ListingId'] == id_1]]
    url_2 =[x for x in df_clean_1['listing_url'][df_clean_1['ListingId'] == id_2]]
    print("Use 1 if the urls are od the same property, 0 if not, 3 for a full description, 2 for ambiguous")
    print(url_1[0])
    print(url_2[0])
    label = int(input())
    if label <= 2:
        labels.append(((id_1,id_2),label))
        print('Saved!')
    else:
        lista = [print(x,':',y,'==',z) for x,y,z in zip(df_clean_1.columns.tolist(),
                                                        df_clean_1[df_clean_1['ListingId'] == id_1].values.tolist()[0],
                                                        df_clean_1[df_clean_1['ListingId'] == id_2].values.tolist()[0])]
        label = int(input())
        labels.append(((id_1,id_2),label))
        print('Saved!')

#Labeling loop
for id_1,id_2 in sample_bottom:
    url_1 =[x for x in df_clean_1['listing_url'][df_clean_1['ListingId'] == id_1]]
    url_2 =[x for x in df_clean_1['listing_url'][df_clean_1['ListingId'] == id_2]]
    print("Use 1 if the urls are od the same property, 0 if not, 3 for a full description, 2 for ambiguous")
    print(url_1[0])
    print(url_2[0])
    label = int(input())
    if label <= 2:
        labels.append(((id_1,id_2),label))
        print('Saved!')
    else:
        lista = [print(x,':',y,'==',z) for x,y,z in zip(df_clean_1.columns.tolist(),
                                                        df_clean_1[df_clean_1['ListingId'] == id_1].values.tolist()[0],
                                                        df_clean_1[df_clean_1['ListingId'] == id_2].values.tolist()[0])]
        label = int(input())
        labels.append(((id_1,id_2),label))
        print('Saved!')


Use 1 if the urls are od the same property, 0 if not, 3 for a full description, 2 for ambiguous
https://www.airbnb.nl/rooms/25463991
https://www.airbnb.nl/rooms/25336232
3
site : AIRBNB == AIRBNB
ListingId : 25463991 == 25336232
Name : karel appel apartment with canal view   terrace == frans hals apartment garden view
postalcode : 1017 EN == 1017 EN
Bedrooms : 3.0 == 1.0
Bathrooms : 1.0 == 1.0
Beds : 5.0 == 1.0
Person_capacity : 6 == 2
Host_first_name : anja == anja
Instant_bookable : 0 == 0
Is_new_listing : 0 == 0
Neighborhood : amsterdam centrum == amsterdam centrum
Lat : 52.3643 == 52.3643
Lng : 4.8886699999999985 == 4.8886699999999985
Picture_url : https://a0.muscache.com/im/pictures/a182b23c-ae21-40cb-a5b0-db79b7b1bb23.jpg?aki_policy=large == https://a0.muscache.com/im/pictures/641eb7a4-0d6d-45fb-9219-b02f1fb739ca.jpg?aki_policy=large
reviews_count : 1.0 == 2.0
Room_type_category : entire_home == entire_home
Room_type : gehele woning appartement == gehele woning appartement
Space_

0
Saved!
Use 1 if the urls are od the same property, 0 if not, 3 for a full description, 2 for ambiguous
https://www.tripadvisor.nl/VacationRentalReview-g188590-d13323008-NDSM_Apartment_11-Amsterdam_North_Holland_Province.html
https://www.tripadvisor.nl/VacationRentalReview-g188590-d12845162-NDSM_Apartment_9-Amsterdam_North_Holland_Province.html
1
Saved!
Use 1 if the urls are od the same property, 0 if not, 3 for a full description, 2 for ambiguous
https://www.airbnb.nl/rooms/48127669
https://www.airbnb.nl/rooms/48127625
0
Saved!
Use 1 if the urls are od the same property, 0 if not, 3 for a full description, 2 for ambiguous
https://www.vrbo.com/nl-nl/vakantiewoning/p8860335?noDates=true
https://www.booking.com/hotel/nl/the-harlemstreet-suites-amsterdam.en-gb.html
0
Saved!
Use 1 if the urls are od the same property, 0 if not, 3 for a full description, 2 for ambiguous
https://www.airbnb.nl/rooms/45646002
https://www.expedia.nl/Nederland.h2540525.Hotelinfo
0
Saved!
Use 1 if the urls are o

In [16]:
# Saving labels as pickle
file_name = "labels.pkl"
open_file = open(file_name, "wb")
pickle.dump(labels, open_file)
open_file.close()
